# Requirements and basic imports

In [ ]:
!pip install -q -U langchain
!pip install -q langchain-community
!pip install -q langchain_groq
!pip install -q langchain_openai
!pip install -q wget

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os, wget

In [ ]:
os.environ['OPENAI_API_KEY'] = ""

# Document Preparation

In [ ]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

wget.download(url, out = filename)

with open(filename, 'r') as file:
    contents = file.read()
print(contents[0:100])

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards th


In [ ]:
############### DOCUMENT LOADER ###############
# !pip install -q langchain
# !pip install -q langchain_community
from langchain_community.document_loaders import TextLoader

loader = TextLoader(filename)
document = loader.load()

In [ ]:
############### TEXT SPLITTER ###############
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size = 100,
                    chunk_overlap = 20,
                    length_function = len)

paragraphs = text_splitter.split_documents(document)

len(paragraphs) # 215

215

# Embeddings and Retrievers

In [ ]:
############### EMBEDDING MODEL ###############

# !pip install -q langchain-huggingface sentence-transformers
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

# !pip install -q langchain-openai
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model = "text-embedding-3-small")

In [ ]:
############### VECTOR STORES ###############

# !pip install langchain-community
# !pip install chromadb
# !pip install faiss-cpu (oppure faiss-gpu)
from langchain_community.vectorstores import Chroma, FAISS

#### METODO 1 ####
""" definisco il 'chroma_db' direttamente dai documenti. Se runno due volte la cella, i chunks vengono
    appesi alla fine, e non posso gestire questo problema. il 'faiss_db' non ha questo problema,
    ad ogni inizializzazione butta via il db precedente """

chroma_db = Chroma.from_documents(paragraphs, embedding_model)
faiss_db = FAISS.from_documents(paragraphs, embedding_model)

#### METODO 2 ####
# !pip install -U langchain-chroma
from langchain_chroma import Chroma
import shutil
""" prima definisco il vector store e poi aggiungo (migliore per gestire il problema di 'chroma_db'). """

COLLECTION_NAME = "mia_collezione_test"
chroma_db = Chroma(embedding_function = embedding_model,
                   collection_name = COLLECTION_NAME)
try: chroma_db.delete_collection() # cancello il db se gia' esiste
except: pass # se non esiste, non succede niente

chroma_db = Chroma.from_documents(documents = paragraphs,
                                  embedding = embedding_model,
                                  collection_name = COLLECTION_NAME)

chroma_db._collection.count() # deve essere esattamente come len(paragraphs)

# test di funzionamento
query = "What is the goal of the Drug and Alchool Policy of our company?"

answer_chroma_db = chroma_db.similarity_search(query, k = 5) # top-k results
answer_faiss_db = faiss_db.similarity_search(query, k = 5)

In [ ]:
############### RETRIEVAL ###############
chroma_retriever = chroma_db.as_retriever(search_kwargs = {"k": 5},
                                          search_type = "similarity")
faiss_retriever = faiss_db.as_retriever(search_kwargs = {"k": 5},
                                        search_type = "similarity")

# test di funzionamento
chroma_retriever.invoke(query)

[Document(id='56ee6171-9cab-406a-ab0e-d4dfdbe38aa7', metadata={'source': 'companyPolicies.txt'}, page_content='Policy Objective: The Drug and Alcohol Policy is established to establish clear expectations and'),
 Document(id='b2431643-45da-431f-94ca-088bfe8c5961', metadata={'source': 'companyPolicies.txt'}, page_content='6.\tDrug and Alcohol Policy'),
 Document(id='e5e8791a-889f-4878-80d7-141795c59665', metadata={'source': 'companyPolicies.txt'}, page_content='expectations and guidelines for the responsible use of drugs and alcohol within the organization.'),
 Document(id='28497bb6-55f9-4c08-8c41-e6019473e662', metadata={'source': 'companyPolicies.txt'}, page_content='Your adherence to this policy is appreciated as it helps to maintain a safe and drug-free workplace'),
 Document(id='37af2ce5-d167-40ad-b59a-557fb0103bc1', metadata={'source': 'companyPolicies.txt'}, page_content='the organization. This policy aims to maintain a safe, healthy, and productive workplace.')]

# RAG with memory

In [ ]:
############### Naive Conversational RAG ###############

# !pip install -q langchain-openai
from langchain_openai import ChatOpenAI
gpt4o = ChatOpenAI(model_name = "gpt-4o-mini")

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {} # questo simula un database dove salvare la storia
def get_session_history(session_id: str):
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

message = [
      ("system", "You are an useful assistant which helps workers about company rules and values."),
      MessagesPlaceholder(variable_name = "chat_history"),
      ("user", "Answer to this query ```{query}```using the context ```{top_context}``` and your memory.")
  ]
prompt = ChatPromptTemplate.from_messages(message)
chain = prompt | gpt4o | StrOutputParser()
chain_with_memory = RunnableWithMessageHistory(
                        chain,
                        get_session_history,
                        history_messages_key = "chat_history",
                        input_messages_key = "query")

def generate_answer(query):
  retrieved = chroma_retriever.invoke(query)
  top_context_list = [retrieved[i].page_content for i in range(len(retrieved))]
  top_context = " ".join(top_context_list)


  return chain_with_memory.invoke({"query": query, "top_context": top_context},
                                  config = {"configurable": {"session_id": "nome_sessione"}})


# Gradio interface

In [ ]:
############### GRADIO ###############

# !pip install gradio
import gradio as gr

# interfaccia base
rag_application = gr.Interface(
    title = "QA Bot",
    description = "Ask any question. The chatbot will try to answer using the provided document.",
    fn = generate_answer,
    allow_flagging = "never",
    inputs = [
        gr.Textbox(label = "Input Query", lines = 20, placeholder = "Type your question here...")
    ],
    outputs = gr.Textbox(label = "Output", lines = 20)
)

# interfaccia avanzata
theme = gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="indigo",
    neutral_hue="slate",
)

with gr.Blocks(theme=theme, title="QA Bot") as rag_application:
    gr.Markdown(
        """
        # 🤖 QA Bot
        ### Ask any question based on the provided documents.
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            input_box = gr.Textbox(
                label="Input Query",
                placeholder="Type your question here...",
                lines=5,
                autofocus=True
            )
            with gr.Row():
                clear_btn = gr.ClearButton(components=[input_box], variant="secondary")
                submit_btn = gr.Button("Submit Question", variant="primary")

        with gr.Column(scale=1):
            output_box = gr.Textbox(
                label="AI Response",
                lines=10,
                interactive=False,
                show_copy_button=True
            )

    submit_btn.click(
        fn=generate_answer,
        inputs=input_box,
        outputs=output_box
    )

    input_box.submit(
        fn=generate_answer,
        inputs=input_box,
        outputs=output_box
    )
rag_application.launch(share = True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(
/tmp/ipython-input-389698688.py:25: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, title="QA Bot") as rag_application:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://43a2ffa495306b2bee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
